<a href="https://colab.research.google.com/github/NilupulNakandala/GrooveOn-Hairstyle-recommendation-/blob/main/GROOVEON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project#

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import pandas as pd  # For data manipulation
import numpy as np





In [17]:
#Assign the dataset path to a variable
data_directory = '/content/drive/MyDrive/SDGP_Project/Dataset'

import os
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

#face shape catogaries
categories = os.listdir(data_directory)

# references - https://towardsdatascience.com/implementing-k-nearest-neighbors-with-scikit-learn-9e4858e231ea

# Initialize lists to store images and labels
images = []
labels = []

# Iterate through each category of images
for category in categories:
    category_path = os.path.join(data_directory, category)
    for image_path in os.listdir(category_path):
        image = cv2.imread(os.path.join(category_path, image_path)) # Read the image using OpenCV
        # Resize the image to a consistent size
        image = cv2.resize(image, (224, 224))  # Adjust size as needed

        # Append the image and its corresponding label to the lists
        images.append(image)
        labels.append(category)

# Convert images to NumPy arrays
images = np.array(images)

#print(images)
#print(labels)

# split the dataset to Training and testing
X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.2) #80% for training and 20% for testing

# Assuming X_train is a list of 3D images (height, width, channels)
X_train_flattened = []
for image in X_train:
    flattened_image = image.flatten()  # Reshape to 1D array
    X_train_flattened.append(flattened_image)

#Creating a Support Vector Machine (SVM) model to regression
model = SVC() # SVM expects only 2D inputs
model.fit(X_train_flattened, Y_train) #


# Save the trained model for future use
#model.save("face_shape_model.pkl")

SVC()

In [19]:
#sklearn library to evaluate model performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Flatten the testing images as well
X_test_flattened = []
for image in X_test:
    flattened_image = image.flatten()
    X_test_flattened.append(flattened_image)

# Make predictions on the testing set
predicted_labels = model.predict(X_test_flattened)

# Calculate accuracy, precision, recall, and confusion matrix
accuracy = accuracy_score(Y_test, predicted_labels)
precision = precision_score(Y_test, predicted_labels, average='weighted')
recall = recall_score(Y_test, predicted_labels, average='weighted')
confusion_matrix = confusion_matrix(Y_test, predicted_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Confusion Matrix:\n", confusion_matrix)

Accuracy: 0.0
Precision: 0.0
Recall: 0.0
Confusion Matrix:
 [[0 0 1 2 0 0 0]
 [0 0 3 0 0 0 0]
 [0 0 0 2 0 0 2]
 [0 0 1 0 0 0 1]
 [0 0 1 0 0 1 0]
 [0 0 0 4 0 0 0]
 [0 0 1 1 0 0 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
#shape prediction
new_image = cv2.imread("/content/drive/MyDrive/SDGP_Project/damnd.jpeg")
new_image = cv2.resize(new_image, (224, 224))  # Resize to match training data
new_features = new_image.flatten()
predicted_label = model.predict([new_features])[0]  # Reshape as a single-sample array
print("Predicted face shape:", predicted_label)
#print(image)

Predicted face shape: oval


In [ ]:
testdatadir = "/content/NewInputData/diamond.jpeg" # test run
testdata = cv2.imread(testdatadir)
print(testdata)

In [ ]:
#different approach

import cv2
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image

train = ImageDataGenerator(rescale=1/255)

train_dataset = train.flow_from_directory(data_directory,target_size=(200,200),batch_size=3, class_mode = 'binary')



train_dataset.class_indices

classess = ['diamond','heart','oblong','oval','round','square','triangle']

cnn=tf.keras.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=64,padding='same',strides=2,kernel_size=3,activation='relu',input_shape=(200,200,3)))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=32,padding='same',strides=2,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(7,activation='softmax'))


cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
cnn.fit(train_dataset,epochs=10)




img = cv2.imread("../input/face-shape-classification/face shape detector/diamond/download (10).jpg")
plt.imshow(img)
plt.show()
img = cv2.resize(img,(200,200))
img = np.reshape(img,[1,200,200,3])
classes = cnn.predict(img)
print(np.argmax(classes))